# Реализация Encoder-Decoder

В этом уроке мы рассмотрим реализацию модели Seq2Seq (Encoder-Decoder) на практике. Реализуем модель саостоятельно с помощью TensorFlow (из готовых модулей будем использовать только LSTM слои) и обучим её на простой задаче машинного перевода с английского на русский.

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [ ]:
import codecs
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0


### Загрузка датасета

В качестве обучающего датасета будем использовать пары коротких английских и русских предложений (источник: http://www.manythings.org/anki/). Возьмём первые 10000 фраз (они отсортированы по длине, так что мы берем самые короткие для простоты).

Для работы этого кода необходимо загрузить файл `rus.txt` в Colab.

Считываем строчки из этого файла, парсим их и помещаем предложения в списки `input_texts` и `target_texts` (входные и выходные предложения соответственно).

In [ ]:
data_fpath = '/content/rus.txt'
max_sentences = 10000

input_texts = []
target_texts = []
lines = codecs.open(data_fpath, 'r', encoding='utf8').readlines()[:max_sentences]
for line in lines:
    input_text, target_text, = line.split('\t')[:2]
    input_texts.append(input_text)
    target_texts.append(target_text)

### Подготовка словарей

Как и раньше, в качестве элемента последовательности будем использовать один символ (а не слово). Это подойдет для нашей простой задачи с короткими предложениями.

Подготовим два словаря (отображения индекса в символ и символа в индекс), и сделаем это для входных текстов (`input_texts`) и выходных (`target_texts`), так как они на разных языках и состоят из разных символов.

Кроме того, нам понадобятся специальные токены для начала и конца цепочки (`<START>`, `<END>`).

In [ ]:
def prepare_vocab(texts):
    vocab = sorted(set(''.join(texts)))
    vocab.append('<START>')
    vocab.append('<END>')
    vocab_size = len(vocab)
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    return vocab_size, char2idx, idx2char

INPUT_VOCAB_SIZE, input_char2idx, input_idx2char = prepare_vocab(input_texts)
TARGET_VOCAB_SIZE, target_char2idx, target_idx2char = prepare_vocab(target_texts)

### Подготовка обучающего датасета

Наша модель будет состоять из двух частей: `Encoder` и `Decoder`. Задача энкодера считать входную цепочку и получить её закодированное представление. А задача декодера по этому закодированному представлению получить выходную цепочку.

Декодер по сути является генератором текста, поэтому используется он аналогично тому, как мы это делали ранее с символьным генератором текста. Отличие только в том, что тут декодер будет получать начальное состояние из энкодера, а в качестве "начала" цепочки будет получать токен `<START>`.

И точно так же, как и в случае с генератором, для обучения декодера в качестве входа и целевого выхода будем использовать одну и ту же цепочку, но сдвинутую на один элемент во времени. В конце Декодер должен предсказать токен `<END>`.

Например, входом и выходом для декодера могут быть такие две цепочки из семи символов (начальный и конечный токен это один символ):

`<START>Привет` --> `Привет<END>`

Таким образом, для обучения `Encoder-Decoder` нам понадоятся три набора цепочек:
 - `encoder_input_seqs` - входы в Encoder
 - `decoder_input_seqs` - входы в Decoder
 - `decoder_target_seqs` - целевые выходы из Decoder (и всей модели Encoder-Decoder)

Сами цепочки будут являться последовательностями целочисленных индексов (полученных с помощью соответствующих словарей).


In [ ]:
input_texts_as_int = [[input_char2idx[c] for c in text] for text in input_texts]
target_texts_as_int = [[target_char2idx[c] for c in text] for text in target_texts]

encoder_input_seqs = [np.array(text) for text in input_texts_as_int]
decoder_input_seqs = []
decoder_target_seqs = []
for target_text in target_texts_as_int:
    decoder_input_seqs.append(np.array([target_char2idx['<START>']] + target_text))
    decoder_target_seqs.append(np.array(target_text + [target_char2idx['<END>']]))

### Паддинг цепочек

Вспомним, что для обучения нам надо использовать батчи, которые состоят из цепочек одинаковой длины. А изначально длина цепочек (как входных, так и выходных) может быть произвольной. Поэтому нам необходимо сделать паддинг -- дополнить все цепочки до некоторой фиксированной длины. Например, с помощью символа пробела `' '`. В качестве длин будем брать максимально возможные среди всех имеющихся цепочек (отдельно для входных, отдельно для выходных).

In [ ]:
max_enc_seq_length = max([len(seq) for seq in encoder_input_seqs])
max_dec_seq_length = max([len(seq) for seq in decoder_input_seqs])

encoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    encoder_input_seqs,
    value=input_char2idx[' '],
    padding='post',
    maxlen=max_enc_seq_length)

decoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    decoder_input_seqs,
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length)

decoder_target_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    decoder_target_seqs,
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length)

### Создание модели

Для создания Encoder-Decoder модели воспользуемся смесьюдвух стилей: реализация моеделй через собственный класс и функциональный API. 

Сами по себе Encoder и Decoder (по отдельности) удобно реализовать в виде кастомных классов (наследованных от `tf.keras.Model`), так как у них может быть какая-то сложная реализация. 

В нашем случае Encoder будет состоять из Embedding слоя и одного LSTM слоя, который будет возвращать финальное состояние после прохода по всей цепочке. В качестве состояния нас интересует и вектор `h` и вектор состояния LSTM `c`. Для него нам понадобится дополнительный флаг `return_state=True`

В Декодере будет Embedding, LSTM и полносвязный слой для генерации финальных ответов (распределение вероятностей по символам). Для прямого распространения (`__call__`) кроме входной цепочки декодер будет получать состояние от энкодера (`init_state`) и будет передавать его в свой LSTM слоя в качестве начального состояния, а возвращать будет предсказанную выходную цепочку (той же длины, return_sequences=True) состояние этого LSTM.

После того, как мы отдельно построили Encoder и Decoder, надо соединить их в Encoder-Decoder. Но так как нам нужно создать несколько входов в модель (отдельно входная цепочка в энкодер, отдельно входная цепочка в декодер) очень удобно сделать это с помощью функционального API. Фходные узлы создаются с помощью `tf.keras.layers.Input`, а затем строим вычислительный граф, используя модели `encoder_model` и `decoder_model`.

Финальная модель -- `seq2seq`

In [ ]:
H_SIZE = 256 # Размерность скрытого состояния LSTM
EMB_SIZE = 256 # размерность эмбеддингов (и для входных и для выходных цепочек)

class Encoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(INPUT_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, return_state=True)
        
    def call(self, x):
        out = self.embed(x)
        _, h, c = self.lstm(out)
        state = (h, c)
        return state

class Decoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(TARGET_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(TARGET_VOCAB_SIZE, activation='softmax')
        
    def call(self, x, init_state):
        out = self.embed(x)
        out, h, c = self.lstm(out, initial_state=init_state)
        out = self.fc(out)
        state = (h, c)
        return out, state

encoder_model = Encoder()
decoder_model = Decoder()

encoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_inputs = tf.keras.layers.Input(shape=(None,))

enc_state = encoder_model(encoder_inputs)
decoder_outputs, _ = decoder_model(decoder_inputs, enc_state)

seq2seq = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Обучение модели

In [ ]:
BATCH_SIZE = 64
EPOCHS = 100

loss = tf.losses.SparseCategoricalCrossentropy()
seq2seq.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
seq2seq.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS)

Train on 10000 samples
Epoch 1/100
10000/10000 [==============================] - 11s 1ms/sample - loss: 0.8402 - accuracy: 0.7942
Epoch 2/100
10000/10000 [==============================] - 3s 301us/sample - loss: 0.5379 - accuracy: 0.8448
Epoch 3/100
10000/10000 [==============================] - 3s 304us/sample - loss: 0.4682 - accuracy: 0.8628
Epoch 4/100
10000/10000 [==============================] - 3s 309us/sample - loss: 0.4199 - accuracy: 0.8772
Epoch 5/100
10000/10000 [==============================] - 3s 303us/sample - loss: 0.3839 - accuracy: 0.8875
Epoch 6/100
10000/10000 [==============================] - 3s 305us/sample - loss: 0.3551 - accuracy: 0.8960
Epoch 7/100
10000/10000 [==============================] - 3s 300us/sample - loss: 0.3306 - accuracy: 0.9031
Epoch 8/100
10000/10000 [==============================] - 3s 307us/sample - loss: 0.3096 - accuracy: 0.9090
Epoch 9/100
10000/10000 [==============================] - 3s 303us/sample - loss: 0.2915 - accuracy: 0.91

### Функция для инференса

Запуск инференса для Encoder-Decoder состоит из последовательного применения энкодера и декодера. 

Сначала прогоняем входную цепочку через энкодер и получаем закодированное представление `state`.

А дальше применяем декодер в похожем режиме, как это было с генератором текста (только теперь передаём `state` в качестве начального состояния). В цикле постепенно генерируем выходную цепочку, подавая в декодер лишь один (текущий) символ и получая один предсказанный (следующий) символ. Начинаем с символа `<START>` и повторяем до тех пор, пока не получим символ `<END>` на выходе или не достигнем лимита по количеству символов в цепочке. Для определения того, какой символ предсказал декодер, просто воспользуемся  функцией `argmax` для выходного распределения (выхода FC слоя).

In [ ]:
def seq2seq_inference(input_seq):
    state = encoder_model(input_seq)

    target_seq = np.array([[target_char2idx['<START>']]])

    decoded_sentence = ''
    while True:
        output_tokens, state = decoder_model(target_seq, state)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_idx2char[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '<END>' or
           len(decoded_sentence) > max_dec_seq_length):
            break

        target_seq = np.array([[sampled_token_index]])

    return decoded_sentence

### Пример инференса

Попробуем инференс Seq2Seq моедли на цепочках из нашего датасета.

In [ ]:
for seq_index in range(0, 20):
    input_seq = encoder_input_seqs[seq_index: seq_index + 1]
    decoded_sentence = seq2seq_inference(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Result sentence:', decoded_sentence)
    print('Target sentence:', target_texts[seq_index])

-
Input sentence: Go.
Result sentence: Иди.<END
Target sentence: Марш!
-
Input sentence: Go.
Result sentence: Иди.<END
Target sentence: Иди.
-
Input sentence: Go.
Result sentence: Иди.<END
Target sentence: Идите.
-
Input sentence: Hi.
Result sentence: Здрасте.<END
Target sentence: Здравствуйте.
-
Input sentence: Hi.
Result sentence: Здрасте.<END
Target sentence: Привет!
-
Input sentence: Hi.
Result sentence: Здрасте.<END
Target sentence: Хай.
-
Input sentence: Hi.
Result sentence: Здрасте.<END
Target sentence: Здрасте.
-
Input sentence: Hi.
Result sentence: Здрасте.<END
Target sentence: Здоро́во!
-
Input sentence: Run!
Result sentence: Бегите!<END
Target sentence: Беги!
-
Input sentence: Run!
Result sentence: Бегите!<END
Target sentence: Бегите!
-
Input sentence: Run.
Result sentence: Бегите!<END
Target sentence: Беги!
-
Input sentence: Run.
Result sentence: Бегите!<END
Target sentence: Бегите!
-
Input sentence: Who?
Result sentence: Кто?<END
Target sentence: Кто?
-
Input sentence: Wow

**[Задание 1]** Добавьте в модель Encoder-Decoder еще один LSTM слой (для увеличения глубины). Сделать это нужно и в Encoder и в Decoder. Скрытое состояние Энкодера необходимо сохранять для **каждого** LSTM слоя и передавать в соответствющий LSTM слой Декодера (из первого в первый, из второго во второй).

**[Задание 2]** Сделайте Encoder в Seq2Seq модели двунаправленным (Bidirectional).

